In [1]:
import mido
import os
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Using TensorFlow backend.
/Users/StephanieBrito/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [51]:
mid = MidiFile('Data/devorame-lalo.mid')  

In [56]:
notes = []

def isPercussion(channel):
    return channel in range(8, 17) or channel in range(35, 82)

for msg in mid:
    if msg.type == 'program_change':
        print(msg)
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

program_change channel=9 program=2 time=0
program_change channel=5 program=0 time=0
program_change channel=3 program=32 time=0
program_change channel=9 program=0 time=0
program_change channel=4 program=62 time=0
program_change channel=0 program=57 time=0
program_change channel=9 program=2 time=0
program_change channel=1 program=53 time=0


In [53]:
# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 8
n_to_predict = 2
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = np.asarray(X[:-300])
y = y[:-300]


In [48]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(len(dummy_y[0]))

ValueError: zero-size array to reduction operation maximum which has no identity

In [41]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=n_prev, activation='relu'))
    model.add(Dense(len(dummy_y[0]), activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [42]:
model = baseline_model()
ll = model.fit(X, dummy_y, 32, 50, verbose=1)

Epoch 1/50
4568/4568 [==============================] - 1s 133us/step - loss: 7.7628 - acc: 0.0508
Epoch 2/50
4568/4568 [==============================] - 0s 52us/step - loss: 3.6433 - acc: 0.1003
Epoch 3/50
4568/4568 [==============================] - 0s 67us/step - loss: 3.5577 - acc: 0.1022
Epoch 4/50
4568/4568 [==============================] - 0s 53us/step - loss: 3.4979 - acc: 0.1020
Epoch 5/50
4568/4568 [==============================] - 0s 46us/step - loss: 3.4499 - acc: 0.1020
Epoch 6/50
4568/4568 [==============================] - 0s 50us/step - loss: 3.4099 - acc: 0.1020
Epoch 7/50
4568/4568 [==============================] - 0s 49us/step - loss: 3.3766 - acc: 0.1020
Epoch 8/50
4568/4568 [==============================] - 0s 48us/step - loss: 3.3491 - acc: 0.1022
Epoch 9/50
4568/4568 [==============================] - 0s 49us/step - loss: 3.3263 - acc: 0.1022
Epoch 10/50
4568/4568 [==============================] - 0s 47us/step - loss: 3.3070 - acc: 0.1022
Epoch 11/50
4568/4

In [43]:
prediction = model.predict(np.array(X_test))
class_labels = np.argmax(prediction, axis=1)
labels = list(set(y))
final_labels = [labels[i] for i in class_labels]
print(final_labels)
# for i in c
# print(list(set(y)))
# print(y[0])
# print(dummy_y[0])
# prediction = np.squeeze(prediction)
# prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
# prediction = [int(i) for i in prediction]

[64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 59, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 59, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 59, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,

In [33]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in final_labels:
    # 147 means note_on
    # 67 is velosity
    print(note)
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('20_epoch_8_note_seq_CE.mid')

68
61
68
61
69
68
63
68
61
60
61
61
68
61
68
61
61
68
61
61
61
68
61
68
60
61
61
68
61
68
60
60
61
61
61
61
68
61
60
68
61
68
61
69
61
63
61
68
61
69
61
68
63
60
61
61
61
61
68
61
68
61
68
61
68
61
68
60
60
61
68
61
60
60
61
61
61
61
68
61
60
68
61
61
68
61
61
60
68
61
68
61
61
68
60
60
61
60
61
61
68
61
68
61
68
61
68
61
60
60
61
60
61
68
61
60
68
61
60
61
61
61
61
68
61
61
68
68
61
68
61
61
60
60
68
61
68
61
60
61
68
60
60
61
60
61
61
68
61
61
68
61
61
68
60
60
61
68
60
60
61
68
61
60
60
60
61
61
68
60
60
61
68
60
68
61
61
61
68
61
68
61
60
61
68
61
68
61
69
68
63
68
61
60
61
61
68
61
68
61
68
61
60
61
61
68
61
68
60
61
61
68
61
68
60
60
61
61
61
61
68
61
60
68
61
68
61
69
61
68
61
68
61
69
61
68
63
60
61
61
61
61
68
61
68
61
68
61
68
61
68
60
60
61
60
68
61
69
61
68
60
61
61
61
60
60
61
60
61
60
61
68
61
68
61
60
60
61
61
61
68
60
61
68
61
60
60
60
61
68
60
61
61
60
61
60
61
60
60
61
